In [1]:
# Working with Dataframes
# -----------------------------------------------------------------------
import pandas as pd

# Path configuration for custom module imports
# -----------------------------------------------------------------------
import sys
sys.path.append('../')  # Adds the parent directory to the path for custom module imports

# Import custom functions to work with databases
# -----------------------------------------------------------------------
from src.support_db import sql_query
from src.support_queries import query

### Data loading

In [2]:
data, columns = sql_query(query)

if data and columns:
    df = pd.DataFrame(data, columns=columns)
else:
    print("No data returned from the query.")

Query performed successfully.
Database connection closed.


Computing average sessions per week and user

In [3]:
# View date to date time
df['view_date'] = pd.to_datetime(df['view_date'])

In [ ]:
# We add a columns for week and year (since we have 21 and 22)
df['week'] = df['view_date'].dt.isocalendar().week
df['year'] = df['view_date'].dt.isocalendar().year

# Group by user_id, year, and week to count sessions per week
weekly_sessions = df.groupby(['user_id', 'year', 'week']).size().reset_index(name='sessions_per_week')

# Group by user_id again to get the average
average_sessions_per_week = weekly_sessions.groupby('user_id')['sessions_per_week'].mean().reset_index()
average_sessions_per_week.rename(columns={'sessions_per_week': 'average_sessions_per_week'}, inplace=True)

---

We calculate new metrics:  

* `average_content_rating`

* `average session duration`

* `percentage of content completed`

* `average sessions per week`

* `click-through rate (CTR)`

In [4]:
df.columns

Index(['user_id', 'user_type', 'country', 'recommendation_system',
       'content_id', 'content_duration', 'content_type', 'content_rating',
       'view_id', 'view_date', 'duration_viewed', 'recommendation_id',
       'recommended_by', 'interaction_id', 'interaction_type'],
      dtype='object')

In [ ]:
df_users = df.groupby("user_id").agg(
    {
    'user_type': 'first', # Get first (and only)
    'recommendation_system': 'first', # Get first (and only)
    'content_id': 'nunique', # Number of different contents
    'content_duration': 'mean', # Sum of minutes
    'content_type': 'nunique', # Number of different content types
    'content_rating': 'mean', # Mean rating
    'view_id': 'nunique', # Number of views
    'duration_viewed': 'mean', # Sum of minutes viewed
    'recommendation_id':'nunique', # Number of recommendations
    'recommended_by': 'first', # Get first (and only)
    'interaction_id': 'nunique', # Number of interactions
    'interaction_type': 'nunique' # Number of distinct interactions
    }
).reset_index()

In [6]:
df_users

,user_id,user_type,recommendation_system,content_id,content_duration,content_type,content_rating,view_id,duration_viewed,recommendation_id,recommended_by,interaction_id,interaction_type
0,1,moderator,Recomendeitor_Plus,5,1304,4,3.26,5,1380,0,None,4,3
1,2,moderator,Recomendeitor,1,23,1,2.2,1,129,0,None,1,1
2,3,moderator,Recomendeitor_Plus,3,210,3,3.5,3,188,0,None,1,1
3,4,admin,Recomendeitor_Plus,2,254,2,2.7,2,119,0,None,1,1
4,5,consumer,Recomendeitor,3,104,3,3.266667,3,199,0,None,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
944,996,content_creator,Recomendeitor_Plus,1,39,1,4.2,1,79,0,None,1,1
945,997,moderator,Recomendeitor_Plus,1,22,1,1.0,1,122,0,None,1,1
946,998,moderator,Recomendeitor_Plus,1,226,1,1.9,1,318,0,None,2,1
947,999,content_creator,Recomendeitor,3,169,3,2.633333,3,404,0,None,1,1
